In [4]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    PT = f.read()
    #PT = Plain text or raw text from book 
print("no. of char in this book:",len(PT))
print(PT[:100])

no. of char in this book: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [5]:
import re
# FT is reformed text 
FT = re.split(r'([,.:;?_!"()\']|--|\s)',PT) # it split each word characters(?/--= etc ) and whitespase( )
FT = [item.strip() for item in FT if item.strip()]# it removes the whitespace
print(FT[:30])
print(len(FT))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [6]:
allwords = sorted(set(FT))
print(len(allwords))

# see altho there is 4690 words but when u sort it by alphbetialy it only takes unique words as The and the is same.

1130


In [7]:
vocab = {token:integer for integer,token in enumerate(allwords)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

print(len(vocab))

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
1130


In [8]:
class STV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = STV1(vocab)

text = """ The ? " ()"""
ids = tokenizer.encode(text)
print(ids)

[93, 10, 1, 3, 4]


In [10]:
tokenizer = STV1(vocab)

ids=[93, 10,1, 3, 4]
text = tokenizer.decode(ids)
print(text)

The?"()


In [11]:
allwords = sorted(set(FT))
allwords.extend(["<|endoftext|>", "<|BOS|>","<|unk|>"])
vocab = {token:integer for integer,token in enumerate(allwords)}
print(len(vocab))

1133


In [12]:
for i, item in enumerate(list(vocab.items())[-3:]):
    print(item)

('<|endoftext|>', 1130)
('<|BOS|>', 1131)
('<|unk|>', 1132)


In [13]:
class STV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = STV2(vocab)

start = "<|BOS|>"
end = "<|endoftext|>"
text1 = "Hello, This is soham."
text11 = start + " " + text1 + " " + end
text2 = "In the sky is good?"
text22 = start + " " + text2 + " " + end

text = text11 + " " + text22

print(text)

<|BOS|> Hello, This is soham. <|endoftext|> <|BOS|> In the sky is good? <|endoftext|>


In [15]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1131, 1132, 5, 97, 584, 1132, 7, 1130, 1131, 55, 988, 1132, 584, 500, 10, 1130]
<|BOS|> <|unk|>, This is <|unk|>. <|endoftext|> <|BOS|> In the <|unk|> is good? <|endoftext|>


### BYTE PAIR ENCODING (BPE)


In [16]:
import importlib
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

In [17]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [18]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [19]:
### CREATING INPUT-TARGET PAIRS

In [20]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [21]:
## for fun lets remove the first 50 tokens
enc_sample = enc_text[50:]
print(len(enc_sample))

5095


In [22]:
context_size = 5 #length of the input
#The context_size of 4 means that the model is trained to look at a sequence of 4 words (or tokens) 
#to predict the next word in the sequence. 
#The input x is the first 4 tokens [1, 2, 3, 4], and the target y is the next 4 tokens [2, 3, 4, 5]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287, 257]
y:      [4920, 2241, 287, 257, 4489]


In [23]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
[290, 4920, 2241, 287, 257] ----> 4489


In [24]:
arr=[290, 4920, 2241, 287, 257,4489]
line = tokenizer.decode(arr)
print(line)

 and established himself in a vill


In [25]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a
 and established himself in a ---->  vill


In [26]:
# There's only one more task before we can turn the tokens into embeddings:implementing an efficient data loader that
# iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which
# can be thought of as multidimensional arrays.

In [27]:
# In particular, we are interested in returning two tensors: an input tensor containing the
# text that the LLM sees and a target tensor that includes the targets for the LLM to predict,

In [28]:
### IMPLEMENTING A DATA LOADER

In [29]:
# Step 1: Tokenize the entire text
    
# Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length

# Step 3: Return the total number of rows in the dataset

# Step 4: Return a single row from the dataset

In [30]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [31]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [32]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [33]:
# example of create_dataloader_v1

dataloader = create_dataloader_v1(raw_text,batch_size=1,max_length=5,stride=1,shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807, 3619]])]


In [34]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [35]:
# if for eg we have 6 tokens

input_ids = torch.tensor([2,3,0,1,4,5])

In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # it will generate 6*3 matrics

In [37]:
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [38]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [39]:
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], grad_fn=<EmbeddingBackward0>)
